In [2]:
#test embedding method on a large dataset can help small dataset

In [3]:
import mne

# Load the saved epochs
epochs = mne.read_epochs('5fepochs.fif')

# Now, loaded_epochs is an Epochs object, and you can perform operations or analysis on it as you would with any other Epochs object.


Reading /Users/gongqianxi/Documents/GitHub/psyco_exp/5fepochs.fif ...
    Found the data of interest:
        t =       0.00 ...    1000.00 ms
        0 CTF compensation matrices available


/var/folders/36/kdqqc2hj0vj0qz_r5mtckf700000gn/T/ipykernel_87940/2557030326.py:4: RuntimeWarning: This filename (5fepochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs('5fepochs.fif')


Not setting metadata
1915 matching events found
No baseline correction applied
0 projection items activated


In [4]:
epochs

Number of events,1915
Events,0: 9591: 2072: 1733: 1984: 1895: 189
Time range,0.000 – 1.000 s
Baseline,off


In [5]:
montage = epochs.get_montage()

In [6]:
#test with embeddings 
#add embeddings
import math
import torch

def positionalencoding3d(d_model, x, y, z):
    if d_model % 6 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with odd dimension (got dim={:d})".format(d_model))
    
    # Calculate the positional encodings for X, Y, and Z separately
    x_encoding = torch.sin(x * (2 ** torch.arange(0, d_model, 3) / d_model))
    y_encoding = torch.cos(y * (2 ** torch.arange(1, d_model, 3) / d_model))
    z_encoding = torch.sin(z * (2 ** torch.arange(2, d_model, 3) / d_model))

    # Combine the encodings
    position_encoding = x_encoding + y_encoding + z_encoding
    
    return position_encoding


ch_names = montage.ch_names
pos = montage.get_positions()

# Specify the dimensionality of the encoding
d_model = 12
embeddings = [] 
# Calculate the positional encoding
for name in ch_names:
    x=pos['ch_pos'][name][0]
    y=pos['ch_pos'][name][1]
    z=pos['ch_pos'][name][2]
    print(name,x,y)
    position_encoded = positionalencoding3d(d_model,x,y,z)
    embeddings.append(position_encoded)

embeddings = torch.stack(embeddings)

Fp1 -0.030902596190571785 0.11458518356084824
Fp2 0.02840948849916458 0.11534630507230759
F3 -0.05180904641747475 0.08668790012598038
F4 0.05027427151799202 0.08743838220834732
C3 -0.06714873015880585 0.023358235135674477
C4 0.06532887369394302 0.023573100566864014
P3 -0.05503823980689049 -0.04421029984951019
P4 0.05363600701093674 -0.044334527105093
F7 -0.0718766301870346 0.07310353219509125
F8 0.07143526524305344 0.07450512051582336
T3 -0.08598208427429199 0.014871643856167793
T4 0.08326136320829391 0.0152581837028265
T5 -0.07445797324180603 -0.042123161256313324
T6 0.07103246450424194 -0.042259979993104935
Fz -0.0012292786268517375 0.09327445179224014
Cz -0.0013741408474743366 0.027617093175649643
Pz -0.001709452481009066 -0.045212991535663605


In [7]:
import mne
import numpy as np

# Suppose epochs.event_id contains mapping from event name to event id
# and event_id is a field in epochs.events array that tells you to which class an epoch belongs.
event_id_mapping = epochs.event_id  # A dictionary mapping of event names to event ids

class_data = []  # Initializing a list to hold the FFT results for all classes while maintaining sequence

for epoch, event in zip(epochs, epochs.events):
    event_id = event[2]  # Extract the event id from the event.
    
    # Perform STFT and append the result to the class_data list
    fft_res = mne.time_frequency.stft(epoch, 128)
    class_data.append(fft_res[:, 1:20, :])

# Convert the list to a numpy array
combined_data = abs(np.array(class_data))

# Reshape to (num_epochs, -1)
combined_data = combined_data.reshape(combined_data.shape[0], -1)
print(combined_data.shape)
# Save the combined data as an npy file
#np.save("combined_data.npy", combined_data)


embeddings_reshaped=embeddings[None,:,:,None].expand(1915,17,4,76)

combined_data_reshaped=combined_data.reshape(1915,17,1,-1)
print(combined_data_reshaped.shape)

withembeddings=np.concatenate((combined_data_reshaped,embeddings_reshaped),axis=2)
withembeddings=withembeddings.reshape(1915,-1)

np.save('gnnbackup/processed/ml_5f.npy', withembeddings)



Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4


In [8]:
# do node feature which is erps 
import torch

rawerps=[]
for event in [0,1,2,3,4,5]:
    finger_tensor = torch.from_numpy(epochs[str(event)].average(picks='all').get_data()) 
    rawerps.append(finger_tensor)
erps_array = np.array(rawerps)

ffterp=[]
for erp in erps_array:
    fft_erp = mne.time_frequency.stft(erp, 128)
    ffterp.append(fft_erp[:, 1:20, :])
    
ffterp_array = abs(np.array(ffterp))
ffterp_array.shape
embeddings_reshaped=embeddings[None,:,:,None].expand(6,17,4,76)
ffterp_array_reshaped=ffterp_array.reshape(6,17,1,-1)
erpwithembeddings=np.concatenate((ffterp_array_reshaped,embeddings_reshaped),axis=2)
nodefeatures_array=erpwithembeddings.reshape(6,-1)
np.save('gnnbackup/processed/ml_5f_node.npy', nodefeatures_array)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4


In [11]:
print(nodefeatures_array.shape)

(6, 6460)


In [3]:
import mne
import numpy as np

# Suppose epochs.event_id contains mapping from event name to event id
# and event_id is a field in epochs.events array that tells you to which class an epoch belongs.
event_id_mapping = epochs.event_id  # A dictionary mapping of event names to event ids

class_data = {class_id: [] for class_id in range(0, 6)}  # Initializing a dictionary to hold the FFT results for each class

for epoch, event in zip(epochs, epochs.events):
    event_id = event[2]  # Extract the event id from the event.
    class_id = None
    
    # Identifying the class_id for the given event_id
    for name, id in event_id_mapping.items():
        if id == event_id:
            class_id = int(name)  # Convert event name to class id (int), assuming event names are "1", "2", "3", etc.
            break
    
    # Perform STFT and append the result to the corresponding class in class_data dictionary.
    if class_id is not None:
        fft_res = mne.time_frequency.stft(epoch, 128)
        class_data[class_id].append(fft_res[:,1:20,:])

# Converting lists in class_data to numpy arrays for easier subsequent processing
for class_id in class_data.keys():
    class_data[class_id] = np.array(class_data[class_id])


Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4
Number of frequencies: 65
Number of time steps: 4


In [ ]:
np.save('gnnbackup/processed/ml_5f.npy', final_edgefeature)

In [5]:
class_data[2].shape

(173, 17, 19, 4)

In [81]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []

# Split each class's data separately to maintain class-wise consistency
for class_id in class_data.keys():
    data = abs(class_data[class_id])
    labels = np.full((len(data),), class_id)
    X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(data, labels, test_size=0.2, random_state=62)
    
    X_train_list.append(X_train_c)
    X_test_list.append(X_test_c)
    y_train_list.append(y_train_c)
    y_test_list.append(y_test_c)

# Concatenate the class-wise data
X_train = np.vstack(X_train_list)
X_test = np.vstack(X_test_list)
y_train = np.hstack(y_train_list)
y_test = np.hstack(y_test_list)

# Reshape X_train and X_test
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Create and train a Logistic Regression model
model = LogisticRegression(max_iter=130, random_state=44)
model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred = model.predict(X_train)

# Calculate the training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.2f}")

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline Accuracy: {accuracy:.2f}")


Training Accuracy: 1.00
Baseline Accuracy: 0.59


/Users/gongqianxi/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
print(embeddings.shape)

torch.Size([17, 4])


In [80]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []

# Split each class's data separately to maintain class-wise consistency
for class_id in class_data.keys():
    data = abs(class_data[class_id])
    embedding=embeddings[None,:, None, None,:].expand(data.shape[0], 17, 19, 4, 4)
    data=np.expand_dims(data, axis=-1)
    data_with_embedding = np.concatenate((data, embedding), axis=-1)
    labels = np.full((len(data),), class_id)
    X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(data_with_embedding, labels, test_size=0.2, random_state=62)
    
    X_train_list.append(X_train_c)
    X_test_list.append(X_test_c)
    y_train_list.append(y_train_c)
    y_test_list.append(y_test_c)

# Concatenate the class-wise data
X_train = np.vstack(X_train_list)
X_test = np.vstack(X_test_list)
y_train = np.hstack(y_train_list)
y_test = np.hstack(y_test_list)

# Reshape X_train and X_test
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Create and train a Logistic Regression model
model = LogisticRegression(max_iter=130, random_state=44)
model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred = model.predict(X_train)

# Calculate the training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.2f}")

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline Accuracy: {accuracy:.2f}")


Training Accuracy: 0.98
Baseline Accuracy: 0.59


/Users/gongqianxi/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [83]:
allfingers_data = np.load("/Users/gongqianxi/Documents/GitHub/psyco_exp/gnnbackup/processed/ml_hands.npy")

In [84]:
allfingers_data.shape

(1296, 2000)